In [ ]:
pip install roboflow

In [ ]:
import cv2

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="CEfThkqYjDhVzpzg2wJO")
project = rf.workspace().project("breast-cancer-roi-n4ssp")
model = project.version(1).model

print(model.predict("/content/drive/MyDrive/1-018.jpg", confidence=50, overlap=50).json())

prediction = model.predict("/content/drive/MyDrive/1-018.jpg", confidence=50, overlap=50).json()
prediction_list = prediction["predictions"]
prediction_dict = prediction_list[0]

#print(type(prediction_list[0]))

roi_x = int(prediction_dict['x'] - prediction_dict['width'] / 2)
roi_y = int(prediction_dict['y'] - prediction_dict['height'] / 2)
roi_width = int(prediction_dict['width'])
roi_height = int(prediction_dict['height'])

image = cv2.imread("/content/drive/MyDrive/1-018.jpg")

roi = image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
print(roi)

cv2.imwrite("roi.jpeg", roi)

# visualizar o prediction
# model.predict("/content/1-126.jpg", confidence=50, overlap=50).save("prediction.jpg")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
import pandas as pd
import re
import cv2
from roboflow import Roboflow
import os
import glob

In [14]:
dataset = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM'

In [15]:
dicom_data = pd.read_csv(dataset + '/csv/dicom_info.csv')

In [16]:
# patient_id, abnormality type, calc type, pathology, image file path

calc_case_description_train_set = pd.read_csv(dataset + '/csv/calc_case_description_train_set.csv',
                                              header=0,
                                              usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"])

calc_case_description_test_set = pd.read_csv(dataset + '/csv/calc_case_description_test_set.csv',
                                             header=0,
                                             usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"])



In [17]:
# patient_id, abnormality type, mass shape, pathology, image file path

mass_case_description_train_set = pd.read_csv(dataset + '/csv/mass_case_description_test_set.csv',
                                              header=0,
                                              usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"])

mass_case_description_test_set = pd.read_csv(dataset + '/csv/mass_case_description_test_set.csv',
                                                                                           header=0,
                                              usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"])



In [22]:
def alterar_nome_da_pasta(nome):
    teste = "Mass-Test_P_00017_LEFT_CC/1.3.6.1.4.1.9590.100.1.2.289610447411344525237308079592285912683/1.3.6.1.4.1.9590.100.1.2.22131189612893294827907969600765582967/000000.dcm"

    padrao = r'.*?/.*?/(.*?)/'
    resultado = re.search(padrao, nome)

    if resultado:
        string_desejada = resultado.group(1)
        print(string_desejada)
        return string_desejada
    else:
        # Caso a expressão regular não encontre um padrão correspondente, pode retornar algo apropriado ou levantar um erro.
        print("Padrão não encontrado para:", nome)
        return None

In [ ]:
mass_case_description_train_set['image file path']


In [ ]:
calc_case_description_train_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

calc_case_description_test_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

mass_case_description_train_set['image file path'] = mass_case_description_train_set['image file path'].apply(alterar_nome_da_pasta)

mass_case_description_test_set['image file path'] = mass_case_description_test_set['image file path'].apply(alterar_nome_da_pasta)


In [ ]:
mass_case_description_train_set['image file path']

In [37]:
def obter_nomes_imagens(diretorio, extensoes=['*.jpg']):
    caminho_completo = os.path.join(diretorio, '*')
    nomes_imagens = []

    for extensao in extensoes:
        caminho_imagens = os.path.join(diretorio, extensao)
        nomes_imagens.extend(glob.glob(caminho_imagens))

    nomes_imagens = [os.path.basename(imagem) for imagem in nomes_imagens]
    return nomes_imagens

def cortar_imagem(nome_pasta):
    rf = Roboflow(api_key="CEfThkqYjDhVzpzg2wJO")
    project = rf.workspace().project("breast-cancer-roi-n4ssp")
    model = project.version(1).model

    # print(model.predict("/content/1-126.jpg", confidence=50, overlap=50).json())

    nome_pasta = dataset + '/jpeg/' + nome_pasta
    nomes_imagens = obter_nomes_imagens(nome_pasta)

    prediction = model.predict(nome_pasta + '/' + nomes_imagens[0], confidence=50, overlap=50).json()
    prediction_list = prediction["predictions"]
    prediction_dict = prediction_list[0]

    roi_x = int(prediction_dict['x'] - prediction_dict['width'] / 2)
    roi_y = int(prediction_dict['y'] - prediction_dict['height'] / 2)
    roi_width = int(prediction_dict['width'])
    roi_height = int(prediction_dict['height'])

    image = cv2.imread(nome_pasta + '/' + nomes_imagens[0])

    roi = image[roi_y:roi_y+roi_height, roi_x:roi_x+roi_width]
    # print(roi)

    cv2.imwrite(nome_pasta + "/roi_" + nomes_imagens[0], roi)

def processar_pastas(diretorio_raiz, dataframe):
    for nome_pasta in dataframe['image file path']:
        pasta_completa = os.path.join(diretorio_raiz, nome_pasta)
        if os.path.isdir(pasta_completa):
            cortar_imagem(nome_pasta)

#Teste

dataset = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM'

# Carregar o dataframe calc_case_description_train_set
calc_case_description_train_set = pd.read_csv(
    dataset + '/csv/calc_case_description_train_set.csv',
    header=0,
    usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"]
)

# Carregar o dataframe calc_case_description_test_set
calc_case_description_test_set = pd.read_csv(
    dataset + '/csv/calc_case_description_test_set.csv',
    header=0,
    usecols=["patient_id", "abnormality type", "calc type", "pathology", "image file path"]
)

# Carregar o dataframe mass_case_description_train_set
mass_case_description_train_set = pd.read_csv(
    dataset + '/csv/mass_case_description_train_set.csv',
    header=0,
    usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"]
)

# Carregar o dataframe mass_case_description_test_set
mass_case_description_test_set = pd.read_csv(
    dataset + '/csv/mass_case_description_test_set.csv',
    header=0,
    usecols=["patient_id", "abnormality type", "mass shape", "pathology", "image file path"]
)
# Caminho para o diretório de treinamento
diretorio_treinamento = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM/csv/calc_case_description_train_set.csv'
#diretorio_treinamento = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM/csv/mass_case_description_train_set.csv'

# Caminho para o diretório de teste
diretorio_teste = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM/csv/calc_case_description_test_set.csv'
#diretorio_teste = '/content/drive/MyDrive/Inteligência Artificial/CBIS-DDSM/csv/mass_case_description_test_set.csv'

# Chamar a função processar_pastas para o diretório de treinamento
processar_pastas(diretorio_treinamento, mass_case_description_train_set)
processar_pastas(diretorio_treinamento, calc_case_description_train_set)

# Chamar a função processar_pastas para o diretório de teste
processar_pastas(diretorio_teste, mass_case_description_test_set)
processar_pastas(diretorio_teste, calc_case_description_test_set)